In [1]:
import sys
sys.path.append("..")
import nof
from automr import guess, dump_mat
from automr.autocas import get_locorb
from pyscf import lib
lib.num_threads(4)

xyz = '''N 0.0 0.0 0.0; N 0.0 0.0 1.1'''
bas = 'def2-svp'
mf = guess.gen(xyz, bas, 0, 0).run().to_rhf()

#mf, unos, noon, nacto, nelecact, ncore, _ = get_uno(mf, thresh=1.99999)

mf, lmos, npair, ndb = get_locorb(mf)

thenof = nof.SOPNOF(mf, npair*2, npair*2)
thenof.verbose = 5
thenof.fcisolver = nof.fakeFCISolver()
thenof.fcisolver.ncore = ndb
thenof.fcisolver.npair = npair
thenof.internal_rotation = True
thenof.mc2step()

System: uname_result(system='Linux', node='yoga', release='5.10.16.3-microsoft-standard-WSL2', version='#1 SMP Fri Apr 2 22:23:49 UTC 2021', machine='x86_64')  Threads 4
Python 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
numpy 1.20.3  scipy 1.7.1
Date: Sun Apr 17 21:11:49 2022
PySCF version 2.0.1
PySCF path  /home/wsr/pyscf
GIT HEAD (branch master) f1ec3c1786b236cf63c5d16e85dd579d6567f9b3

[ENV] PYSCF_EXT_PATH /home/wsr/dh:/home/wsr/gvb:
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 14
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 N      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr
[INPUT]  2 N      0.000000000000   0.000000000000   1.100000000000 AA    0.000000000000   0.000000000000   2.078698737022 Bohr

nuclear repulsion = 23.5724393955273
number of shells = 12
number of NR 

(-108.92790054737686,
 -13.750382410239173,
 None,
 array([[ 0.701804,  0.700243,  0.192301, -0.194336,  0.107873,  0.000014,  0.000005,  0.000009,  0.000019, -0.201072,  0.186576,  0.076664, -0.179067,
          0.      , -0.000001, -0.069266,  0.000012,  0.000026,  0.      , -0.      , -0.      ,  0.      ,  0.      ,  0.      , -0.041864,  0.000002,
          0.00001 ,  0.085676],
        [-0.019106, -0.024292,  0.375807, -0.421941,  0.222496,  0.00003 ,  0.000011,  0.000036,  0.000076, -0.881513,  1.149688,  0.228878, -0.939636,
          0.000002, -0.000002, -0.670781,  0.000101,  0.000278, -0.      , -0.      , -0.      ,  0.      ,  0.      ,  0.      , -0.542478,  0.000019,
          0.00008 ,  0.574595],
        [ 0.005033,  0.014434,  0.299449, -0.376252, -0.108035, -0.000008, -0.000018,  0.000034,  0.000069,  0.024836, -0.866478, -0.431179,  0.074048,
          0.000003, -0.000014,  8.250773, -0.001092, -0.00412 , -0.      ,  0.000001,  0.      , -0.      , -0.      , -0.000

In [5]:
Pi = thenof.fcisolver.nof.Pi[4:10,4:10]
Pi

array([[ 0.996938,  0.      ,  0.      ,  0.      ,  0.      , -0.055251],
       [ 0.      ,  0.96785 ,  0.      ,  0.      , -0.176398,  0.      ],
       [ 0.      ,  0.      ,  0.96785 , -0.176398,  0.      ,  0.      ],
       [ 0.      ,  0.      , -0.176398,  0.03215 ,  0.      ,  0.      ],
       [ 0.      , -0.176398,  0.      ,  0.      ,  0.03215 ,  0.      ],
       [-0.055251,  0.      ,  0.      ,  0.      ,  0.      ,  0.003062]])

In [6]:
Delta = thenof.fcisolver.nof.Delta[4:10,4:10]
Delta

array([[0.993885, 0.      , 0.      , 0.      , 0.      , 0.003053],
       [0.      , 0.936734, 0.      , 0.      , 0.031116, 0.      ],
       [0.      , 0.      , 0.936734, 0.031116, 0.      , 0.      ],
       [0.      , 0.      , 0.031116, 0.001034, 0.      , 0.      ],
       [0.      , 0.031116, 0.      , 0.      , 0.001034, 0.      ],
       [0.003053, 0.      , 0.      , 0.      , 0.      , 0.000009]])

In [10]:
f = thenof.fcisolver.nof.mo_occ[4:10]
f

array([0.996938, 0.96785 , 0.96785 , 0.03215 , 0.03215 , 0.003062])

In [11]:
from numpy import einsum
a = 2*einsum('q,p->qp', f, f) - 2*Delta
b = -einsum('q,p->qp', f, f) + Delta + Pi
a

array([[0.      , 1.929773, 1.929773, 0.064103, 0.064103, 0.      ],
       [1.929773, 0.      , 1.873467, 0.062233, 0.      , 0.005927],
       [1.929773, 1.873467, 0.      , 0.      , 0.062233, 0.005927],
       [0.064103, 0.062233, 0.      , 0.      , 0.002067, 0.000197],
       [0.064103, 0.      , 0.062233, 0.002067, 0.      , 0.000197],
       [0.      , 0.005927, 0.005927, 0.000197, 0.000197, 0.      ]])

In [12]:
from pyscf import mcscf
mf2 = mcscf.CASSCF(mf,6,6)
mf2.kernel()


******** <class 'pyscf.mcscf.mc1step.CASSCF'> ********
CAS (3e+3e, 6o), ncore = 4, nvir = 18
max_cycle_macro = 50
max_cycle_micro = 4
conv_tol = 1e-07
conv_tol_grad = None
orbital rotation max_stepsize = 0.02
augmented hessian ah_max_cycle = 30
augmented hessian ah_conv_tol = 1e-12
augmented hessian ah_linear dependence = 1e-14
augmented hessian ah_level shift = 1e-08
augmented hessian ah_start_tol = 2.5
augmented hessian ah_start_cycle = 3
augmented hessian ah_grad_trust_region = 3
kf_trust_region = 3
kf_interval = 4
ci_response_space = 4
ci_grad_trust_region = 3
with_dep4 0
natorb = False
canonicalization = True
sorting_mo_energy = False
ao2mo_level = 2
chkfile = /home/wsr/pyNOF/examples/tmpk86b2w6z
max_memory 4000 MB (current use 194 MB)
internal_rotation = False
******** <class 'pyscf.fci.direct_spin1.FCISolver'> ********
max. cycles = 50
conv_tol = 1e-08
davidson only = False
linear dependence = 1e-10
level shift = 0.001
max iter space = 12
max_memory 4000 MB
nroots = 1
pspace_si

(-108.98903481465013,
 -13.696722978866546,
 array([[ 0.966527, -0.000001,  0.      , -0.      ,  0.000001,  0.000001,  0.000001,  0.049208, -0.002673,  0.001436, -0.      ,  0.      ,  0.000001,
          0.003907,  0.024363, -0.012379,  0.000853, -0.012472, -0.024487,  0.      ],
        [-0.000001, -0.101305,  0.043078,  0.000106,  0.042836,  0.055976, -0.004236, -0.      , -0.      ,  0.      ,  0.005009,  0.003679,  0.03778 ,
         -0.      , -0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.008678],
        [ 0.      ,  0.043078, -0.038458,  0.00018 ,  0.006768, -0.042978,  0.002131,  0.      ,  0.      , -0.      , -0.000229, -0.002873, -0.019216,
         -0.      ,  0.      , -0.      ,  0.      , -0.      , -0.      , -0.004411],
        [-0.      ,  0.000106,  0.00018 , -0.008325, -0.001563,  0.000695,  0.000017, -0.      ,  0.      ,  0.      ,  0.013417, -0.006837, -0.000215,
          0.      ,  0.      ,  0.      , -0.      , -0.      ,  0.      , -0.00001 ],


In [23]:
rdm2 = mf2.fcisolver.make_rdm12(mf2.ci, 6, 6)[1]
rdm2[0,0,1,1], rdm2[0,0,2,2], rdm2[0,0,3,3]

(3.8537407896846383, 3.8536756753600567, 0.11066533982163528)

In [24]:
rdm2_gvb = thenof.fcisolver.make_rdm12(None,6,6)[1]
rdm2_gvb[0,0,1,1], rdm2_gvb[0,0,2,2], rdm2_gvb[0,0,3,3]

(3.8595459609935165, 3.8595459724464174, 0.12820603807256178)